In [ ]:
!pip install kagglehub --quiet
!pip install tensorflow --quiet
!pip install scikit-learn --quiet
!pip install pandas matplotlib seaborn opencv-python --quiet


In [ ]:
import kagglehub

# Download dataset
path = kagglehub.dataset_download("kausthubkannan/poultry-diseases-detection")

print("✅ Dataset downloaded.")
print("Path to dataset files:", path)


✅ Dataset downloaded.
Path to dataset files: /kaggle/input/poultry-diseases-detection


In [ ]:
import os

dataset_path = "/kaggle/input/poultry-diseases-detection"

print("Contents of dataset folder:")
print(os.listdir(dataset_path))


Contents of dataset folder:
['poultry_diseases']


In [ ]:
# Update dataset path to include 'poultry_diseases'
dataset_path = os.path.join(dataset_path, "poultry_diseases")

# List the subfolders (classes)
class_folders = os.listdir(dataset_path)
print("Classes:", class_folders)


Classes: ['salmo', 'healthy', 'ncd', 'cocci']


In [ ]:
import pandas as pd

image_paths = []
labels = []

for label in class_folders:
    folder_path = os.path.join(dataset_path, label)
    for img_file in os.listdir(folder_path):
        image_paths.append(os.path.join(folder_path, img_file))
        labels.append(label)

df = pd.DataFrame({
    "image_path": image_paths,
    "label": labels
})

print(df.head())
print("\nTotal images collected:", len(df))


                                          image_path  label
0  /kaggle/input/poultry-diseases-detection/poult...  salmo
1  /kaggle/input/poultry-diseases-detection/poult...  salmo
2  /kaggle/input/poultry-diseases-detection/poult...  salmo
3  /kaggle/input/poultry-diseases-detection/poult...  salmo
4  /kaggle/input/poultry-diseases-detection/poult...  salmo

Total images collected: 6812


In [ ]:
df['label'].value_counts()


,count
label,
salmo,2276
cocci,2103
healthy,2057
ncd,376


In [ ]:
from sklearn.model_selection import train_test_split

# First split into train+val and test (15% test)
train_val_df, test_df = train_test_split(df, test_size=0.15, stratify=df['label'], random_state=42)

# Then split train+val into train and validation (18% val)
train_df, val_df = train_test_split(train_val_df, test_size=0.18, stratify=train_val_df['label'], random_state=42)

print(f"Train samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Test samples: {len(test_df)}")


Train samples: 4747
Validation samples: 1043
Test samples: 1022


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create ImageDataGenerator instances
train_datagen = ImageDataGenerator(
    rescale=1./255,          # Normalize pixel values
    horizontal_flip=True,    # Example augmentation
    rotation_range=20,
    zoom_range=0.15,
    shear_range=0.15,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)  # Just normalization for val and test

# Helper function to create generator from dataframe
def create_generator(df, datagen, batch_size=32):
    return datagen.flow_from_dataframe(
        dataframe=df,
        x_col='image_path',
        y_col='label',
        target_size=(224,224),
        batch_size=batch_size,
        class_mode='categorical'  # because labels are categories
    )

batch_size = 32

train_generator = create_generator(train_df, train_datagen, batch_size)
val_generator = create_generator(val_df, val_test_datagen, batch_size)
test_generator = create_generator(test_df, val_test_datagen, batch_size)


Found 4747 validated image filenames belonging to 4 classes.
Found 1043 validated image filenames belonging to 4 classes.
Found 1022 validated image filenames belonging to 4 classes.


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Load the base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x)  # 4 classes

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Summary
model.summary()


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size and batch size
image_size = (224, 224)
batch_size = 32

# Create data generators with rescaling and augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,          # Normalize pixel values
    rotation_range=20,       # Random rotation for augmentation
    zoom_range=0.2,          # Random zoom
    horizontal_flip=True,    # Random horizontal flip
    validation_split=0.18    # Split some data for validation
)

# Validation data generator only rescales
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.18)

# Prepare generators to load images from directory
train_gen = train_datagen.flow_from_directory(
    directory="/root/.cache/kagglehub/datasets/kausthubkannan/poultry-diseases-detection/versions/1/poultry_diseases",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    directory="/root/.cache/kagglehub/datasets/kausthubkannan/poultry-diseases-detection/versions/1/poultry_diseases",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)


In [ ]:
# Train the model
history = model.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // train_gen.batch_size,
    validation_data=val_gen,
    validation_steps=val_gen.samples // val_gen.batch_size,
    epochs=10
)


In [ ]:
model.save("poultry_disease_model.keras")



In [ ]:
from tensorflow.keras.models import load_model

model = load_model("poultry_disease_model.keras")
print("Model loaded successfully!")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
model.save('/content/drive/MyDrive/poultry_disease_model.keras')


In [ ]:
from google.colab import drive
drive.flush_and_unmount()



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create ImageDataGenerator for rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# Recreate the test generator
test_gen = test_datagen.flow_from_dataframe(
    test_df,
    x_col="image_path",
    y_col="label",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    shuffle=False
)


In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_gen)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


In [1]:
train_gen.class_indices


NameError: name 'train_gen' is not defined